In [5]:
import requests
from urllib.parse import urlencode

import json
import pandas as pd

BASE_URL = 'https://api.gotriple.eu/'

In [2]:
params = {
    'q': 'Maternity and Pregnancy',
    'fq': {
        'type': 'typ_article',
        'has_pdf': 'true',
        'in_language': 'en'
    },
    'include_duplicates': 'false',
    'aggs': {

    },
    'sort': '', # name, publication_date, most_recent --> name:desc
    'page': 1,
    'size': 10,
}

In [3]:
def build_api_url(params):
    cleaned_params = {}
    for key, value in params.items():
        if value:
            if isinstance(value, dict):
                cleaned_params[key] = urlencode(value).replace('&', ';')
            else:
               cleaned_params[key] = value 
    return urlencode(cleaned_params)

In [ ]:
objects_list = []
endpoint = 'documents'
url = f'{BASE_URL}{endpoint}?{build_api_url(params)}'
while True:
    response = requests.get(url)
    if response.ok:
        objects_list.extend(response.json()['hydra:member'])
        # print(response.json()['hydra:totalItems'])
        # print(response.json()['hydra:member'])
        # print(response.json()['hydra:view']['hydra:next'])
        # print(response.json()['hydra:view']['hydra:last'])
        print(url)
    if not response.json()['hydra:view'].get('hydra:next'):
        break
    else:
        url = BASE_URL + response.json()['hydra:view']['hydra:next'][1:]
print(len(objects_list))

In [ ]:
with open('gotriple_api_test_output.json', 'w', encoding='utf-8') as jfile:
    json.dump(objects_list, jfile, indent=4, ensure_ascii=False)

In [3]:
with open('gotriple_api_test_output.json', 'r', encoding='utf-8') as jfile:
    objects_list = json.load(jfile)

In [6]:
to_df_lst = []
for elem in objects_list:
    row = {}
    row['id'] = elem['id']

    for e in elem['abstract']:
        if e['lang'] == 'en':
            row['abstract'] = e['text']
            break

    for e in elem['headline']:
            if e['lang'] == 'en':
                row['title'] = e['text']
                break
    
    row['date_published'] = elem['date_published']
    row['doi'] = ' | '.join(elem['doi'])
    row['keywords'] = ' | '.join([e['text'] for e in elem['keywords']])
    row['triple_voc'] = ' | '.join([e['uri'] for e in elem['knows_about']])
    row['pdf_url'] = ' | '.join(elem['url'])
    row['authors'] = ' | '.join([e['fullname'] for e in elem['author']])
    row['triple_disciplines'] = ' | '.join([f"{e['id']}:{e['confidence']}" for e in elem['topic']])
    row['original_languages'] = ' | '.join(elem['original_languages'])
    row['published'] = ' | '.join(elem['mentions'])
    to_df_lst.append(row)

In [10]:
df = pd.DataFrame.from_dict(to_df_lst)
df.to_excel('gotriple_api_test_output.xlsx')